# Регулярные выражение

## Введение

**Регулярные выражения** (или regex) – это последовательности символов, которые образуют шаблоны поиска.

**Основные возможности регулярных выражений в Python:**
- поиск и извлечение данных
- валидация данных
- а также использование шаблонов для замены, разделения текста

**Преимущества использования регулярных выражений:**
- **Гибкость**: Регулярные выражения предоставляют мощные инструменты для поиска и обработки текста, позволяя работать с самыми разнообразными шаблонами.
- **Универсальность**: Они применимы во многих языках программирования и текстовых редакторах, что делает их полезным инструментом для разработчиков и администраторов.
- **Эффективность**: Правильное использование регулярных выражений позволяет производить поиск и обработку текста с высокой скоростью и эффективностью.

<img src='./img/regex_intro.png'>

In [2]:
import re
import pandas as pd

## Модуль re

Основные методы и функции модуля `re`:
- `re.match()`
- `re.search()`
- `re.findall()`
- `re.sub()`

### re.match()

Функция находит совпадение с начала строки. Например найдем по шаблону дату-время

In [3]:
pattern = r'\d\d-\d\d-\d\d\d\d \d\d:\d\d'
re.compile(pattern).match('31-12-2024 23:59')

<re.Match object; span=(0, 16), match='31-12-2024 23:59'>

и ничего не находит, если искомое вхождение в середине строки

In [4]:
pattern = r'\d\d-\d\d-\d\d\d\d \d\d:\d\d'
re.compile(pattern).match('Сейчас 31-12-2024 23:59')

### re.search()

находит вхождение шаблона в любом месте строки, но только один раз

In [5]:
pattern = r'\d\d-\d\d-\d\d\d\d \d\d:\d\d'
re.compile(pattern).search('Сейчас 31-12-2024 23:59, скоро новый год 01-01-2025 00:00')

<re.Match object; span=(7, 23), match='31-12-2024 23:59'>

### re.findall()
находит все непересекающиеся вхождения шаблона. Возвращает список

In [6]:
pattern = r'\d\d-\d\d-\d\d\d\d \d\d:\d\d'
re.compile(pattern).findall('Сейчас 31-12-2024 23:59, скоро новый год 01-01-2025 00:00')

['31-12-2024 23:59', '01-01-2025 00:00']

### re.sub()
заменяет найденные вхождения на указанное значение

In [7]:
pattern = r'\d\d-\d\d-\d\d\d\d \d\d:\d\d'
re.compile(pattern).sub('00-00-0000 00:00', 'Проверяем 31-12-2024 23:59 как работает замена 01-01-2025 00:00')

'Проверяем 00-00-0000 00:00 как работает замена 00-00-0000 00:00'

Вариант синтаксиса

In [8]:
re.sub(pattern, '00-00-0000 00:00', 'Проверяем 31-12-2024 23:59 как работает замена 01-01-2025 00:00')

'Проверяем 00-00-0000 00:00 как работает замена 00-00-0000 00:00'

### Флаги в регулярных выражениях


### re.IGNORECASE

Учитывать регистр или игнорировать. Пример без флага

In [9]:
pattern = r'[А-ЯЁ]+'
re.compile(pattern).findall('Проверяем как работает ВЫБОР рЕгИсТрА')

['П', 'ВЫБОР', 'Е', 'И', 'Т', 'А']

С флагом

In [10]:
pattern = r'[А-ЯЁ]+'
re.compile(pattern, re.IGNORECASE).findall('Проверяем как работает ВЫБОР рЕгИсТрА')

['Проверяем', 'как', 'работает', 'ВЫБОР', 'рЕгИсТрА']

### re.X

Возможность разбивать регулярное выражение на строки и добавлять комментарии. Внимание – в этом случае все пробелы в выражении должны быть указаны явно с помощью \s

In [11]:
pattern = r'''
    \d\d-\d\d-\d\d\d\d\s    # это дата
    \d\d:\d\d               # это время
    '''
re.compile(pattern, re.X).findall('Сегодня 31-12-2024 23:59 скоро 01-01-2025 00:00')

['31-12-2024 23:59', '01-01-2025 00:00']

## Основные конструкции

### Интервалы символов

In [12]:
pattern = r'[А-ЯЁ]+'
re.compile(pattern).findall('Проверяем 31-12-2024 23:59 как работает ВЫБОР')

['П', 'ВЫБОР']

### Указание количества повторяющихся символов

In [13]:
pattern = r'\d{2}-\d{2}-\d{4} \d{2}:\d{2}'
re.compile(pattern).findall('Сейчас 31-12-2024 23:59, скоро новый год 01-01-2025 00:00')

['31-12-2024 23:59', '01-01-2025 00:00']

### Группировка

In [14]:
pattern = r'(\d{2}-\d{2}-\d{4}) (\d{2}:\d{2})'
re.compile(pattern).findall('Сейчас 31-12-2024 23:59, скоро новый год 01-01-2025 00:00')

[('31-12-2024', '23:59'), ('01-01-2025', '00:00')]

### Именованные группы

In [15]:
pattern = r'(?P<date>\d{2}-\d{2}-\d{4}) (?P<time>\d{2}:\d{2})'
re.compile(pattern).search('Сейчас 31-12-2024 23:59').groupdict()

{'date': '31-12-2024', 'time': '23:59'}

Несколько именованных групп можно извлечь с помощью итератора finditer

In [16]:
[print(x.groupdict()) for x in re.compile(pattern).finditer(
    'Сейчас 31-12-2024 23:59, скоро новый год 01-01-2025 00:00')];

{'date': '31-12-2024', 'time': '23:59'}
{'date': '01-01-2025', 'time': '00:00'}


### Предварительное и обратное условие поиска

In [17]:
pattern = r'([а-яА-ЯёЁ\s]+) (?=\=)'
re.compile(pattern).findall('Текст до знака = и после знака')

['Текст до знака']

In [18]:
pattern = r'(?<=\=) ([а-яА-ЯёЁ\s]+)'
re.compile(pattern).findall('Текст до знака = и после знака')

['и после знака']

### Повторный поиск группы по номеру

Найдем слова из первой группы, которые повторяются через запятую

In [19]:
pattern = r'(\b[а-яё]+\b),\s\1'
re.compile(pattern, re.I).findall('Течёт, течёт — не вытечет, бежит, бежит — не выбежит, летел, летел и улетел')


['Течёт', 'бежит', 'летел']

### Поиск указанных именованных групп

Аналогичная задача, но с именованной группой

In [20]:
pattern = r'(?P<find_double>\b[а-яё]+\b),\s(?P=find_double)'
re.compile(pattern, re.I).findall('Течёт, течёт — не вытечет, бежит, бежит — не выбежит, летел, летел и улетел')

['Течёт', 'бежит', 'летел']

## Примеры применения

Выделить только коды

In [21]:
example = """
    Коды доступа:
    Код 1: ABC123
    Код 2: XYZ456
    Код 3: DEF789
    Код 4: GHI101
"""

pattern = 

{'code': 'ABC123'}
{'code': 'XYZ456'}
{'code': 'DEF789'}
{'code': 'GHI101'}


Выделить имя файла и расширение

In [22]:
example = """
archive.tar.gz
report.final.pdf
image.v2.0.jpg
data.backup.2024-04-01.csv
presentation.final.draft.pptx
"""

pattern = 

{'name': 'archive.tar', 'extension': 'gz'}
{'name': 'report.final', 'extension': 'pdf'}
{'name': 'image.v2.0', 'extension': 'jpg'}
{'name': 'data.backup.2024-04-01', 'extension': 'csv'}
{'name': 'presentation.final.draft', 'extension': 'pptx'}


Распарсить афишу

In [25]:
example ="""
Информация о спектакле:
Название: "Война и мир"
Режиссер: Лев Толстой
Год выпуска: 1869
Жанр: Роман-эпопея

Информация о фильме:
Название: "The Shawshank Redemption"
Режиссер: Фрэнк Дарабонт
Год выпуска: 1994
Жанр: Драма

Информация о фильме:
Название: "Кин-дза-дза"
Режиссер: Георгий Данелия
Год выпуска: 1986
Жанр: Трагикомедия
"""

pattern = 

,type,title,director,year,genre
0,спектакле,"""Война и мир""",Лев Толстой,1869,Роман-эпопея
1,фильме,"""The Shawshank Redemption""",Фрэнк Дарабонт,1994,Драма
2,фильме,"""Кин-дза-дза""",Георгий Данелия,1986,Трагикомедия


Все сообщения, кроме информационных

In [23]:
example = """
Предупреждение: Недостаточно памяти
Ошибка: Системный сбой
Информация: Подключение к базе данных
Предупреждение: Потеря соединения
Ошибка: Файл не найден
Информация: Пицца готова
"""

pattern = 

{'alert': 'Недостаточно памяти'}
{'alert': 'Системный сбой'}
{'alert': 'Потеря соединения'}
{'alert': 'Файл не найден'}


Цвета без восклицательного знака

In [24]:
example = """
синий зеленый! черный! красный фиолетовый оранжевый!
"""

pattern = 

{'color': 'синий'}
{'color': 'красный'}
{'color': 'фиолетовый'}


или полный разбор

In [25]:
example = """
синий зеленый! черный! красный фиолетовый оранжевый!
"""

pattern = 

{'color': 'синий', 'marked_color': None}
{'color': None, 'marked_color': 'зеленый'}
{'color': None, 'marked_color': 'черный'}
{'color': 'красный', 'marked_color': None}
{'color': 'фиолетовый', 'marked_color': None}
{'color': None, 'marked_color': 'оранжевый'}


,color,marked_color
0,синий,0
1,0,зеленый
2,0,черный
3,красный,0
4,фиолетовый,0
5,0,оранжевый


Распарсить заказы – номер, статус, сумма

In [26]:
example = """
    Заказы:
    Заказ №1: Статус - Выполнен, Сумма - $50
    Заказ №2: Статус - Отменен, Сумма - $30
    Заказ №3: Статус - В обработке, Сумма - $70
    Заказ №4: Статус - Отменен, Сумма - $238
    Заказ №5: Статус - Отменен, Сумма - $2.5
"""
pattern = 

{'order_id': '1', 'status': 'Выполнен', 'amount': '50'}
{'order_id': '2', 'status': 'Отменен', 'amount': '30'}
{'order_id': '3', 'status': 'В обработке', 'amount': '70'}
{'order_id': '4', 'status': 'Отменен', 'amount': '238'}
{'order_id': '5', 'status': 'Отменен', 'amount': '2.5'}


Вариант – каждая сумма в своей колонке в зависимости от статуса

In [28]:
example = """
    Заказы:
    Заказ №1: Статус - Выполнен, Сумма - $50
    Заказ №2: Статус - Отменен, Сумма - $30
    Заказ №3: Статус - В обработке, Сумма - $70
    Заказ №4: Статус - Отменен, Сумма - $238
    Заказ №5: Статус - Отменен, Сумма - $2.5
"""
pattern = 

{'order_id': '1', 'process_amount': None, 'complete_amount': '50', 'cancel_amount': None}
{'order_id': '2', 'process_amount': None, 'complete_amount': None, 'cancel_amount': '30'}
{'order_id': '3', 'process_amount': '70', 'complete_amount': None, 'cancel_amount': None}
{'order_id': '4', 'process_amount': None, 'complete_amount': None, 'cancel_amount': '238'}
{'order_id': '5', 'process_amount': None, 'complete_amount': None, 'cancel_amount': '2.5'}


Выделение числа и единиц измерения

In [30]:
text = """
около 5 минут
целых 146 дней
1 ГОД
порядка 3-х сек
в течение 2х месяцев
идут уже 2е сутки как отключили интернет
"""

pattern =

,value,units
0,5,минут
1,146,дней
2,1,ГОД
3,3,сек
4,2,месяцев
5,2,сутки


Выбрать только фамилии

In [92]:
text = """
А.А.Тарковский
реж.Ф.Феллини
режиссер Квентин Тарантино
Ли Джон-хо
Билли Б. Торнтон
Гильермо дель Торо
"""

pattern = 

['Тарковский', 'Феллини', 'Тарантино', 'Джон-хо', 'Торнтон', 'Торо']

In [119]:
text = """
Джон Ли Хукер, Фрэнк Синатра, певец И. Д. Кобзон, Муслим Магомет оглы Магомаев, Леди Гага, Мадонна, Т.Тернер
"""

pattern = 

['Хукер', 'Синатра', 'Кобзон', 'Магомаев', 'Гага', 'Мадонна', 'Тернер']

In [31]:
# Найдем всех участников истории про Джека
text = """
Вот дом, который построил Джек.
А это пшеница,
Которая в тёмном чулане хранится
В доме, который построил Джек.
А это весёлая птица-синица,
Которая часто ворует пшеницу,
Которая в тёмном чулане хранится
В доме, который построил Джек.
Вот кот,
Который пугает и ловит синицу,
Которая часто ворует пшеницу,
Которая в тёмном чулане хранится
В доме, который построил Джек.
Вот пёс без хвоста,
Который за шиворот треплет кота,
Который пугает и ловит синицу,
Которая часто ворует пшеницу,
Которая в тёмном чулане хранится
В доме, который построил Джек.
А это корова безрогая,
Лягнувшая старого пса без хвоста,
Который за шиворот треплет кота,
Который пугает и ловит синицу,
Которая часто ворует пшеницу,
Которая в тёмном чулане хранится
В доме, который построил Джек.
А это старушка, седая и строгая,
Которая доит корову безрогую,
Лягнувшую старого пса без хвоста,
Который за шиворот треплет кота,
Который пугает и ловит синицу,
Которая часто ворует пшеницу,
Которая в тёмном чулане хранится
В доме, который построил Джек.
А это ленивый и толстый пастух,
Который бранится с коровницей строгою,
Которая доит корову безрогую,
Лягнувшую старого пса без хвоста,
Который за шиворот треплет кота,
Который пугает и ловит синицу,
Которая часто ворует пшеницу,
Которая в тёмном чулане хранится
В доме, который построил Джек.
Вот два петуха,
Которые будят того пастуха,
Который бранится с коровницей строгою,
Которая доит корову безрогую,
Лягнувшую старого пса без хвоста,
Который за шиворот треплет кота,
Который пугает и ловит синицу,
Которая часто ворует пшеницу,
Которая в тёмном чулане хранится
В доме, который построил Джек.
"""

pattern = 

['дом',
 'пшеница',
 'весёлая птица-синица',
 'кот',
 'пёс без хвоста',
 'корова безрогая',
 'старушка',
 'ленивый и толстый пастух',
 'два петуха']

In [347]:
text = """
Гермомешок Helios 10 л	352,00 руб.
Гермомешок Helios 120 л (HS-DB-1204385-H)	1 718,00 руб.
Гермомешок Helios 15 л (HS-DB-152562-B)	988,00 руб.
Гермомешок Helios 15 л (HS-DB-152562-H)	988,00 руб.
Гермомешок Helios 15 л (HS-DB-152562-Y)	988,00 руб.
Гермомешок Helios 160 л (HS-DB-160-B)	2 011,00 руб.
Гермомешок Helios 160 л (HS-DB-160-Y)	2 011,00 руб.
Гермомешок Helios 20 л	410,00 руб.
Гермомешок Helios 30 л (HS-DB-303070-H)	1 132,00 руб.
Гермомешок Helios 30 л (HS-DB-303070-Y)	1 132,00 руб.
Гермомешок Helios 50 л (HS-DB-503369-B)	1 267,00 руб.
Гермомешок Helios 50 л (HS-DB-503369-H)	1 267,00 руб.
Гермомешок Helios 50 л (HS-DB-503369-Y)	1 267,00 руб.
Гермомешок Helios 70 л (HS-DB-7033100-H)	1 457,00 руб.
Гермомешок Helios 70 л (HS-DB-7033100-Y)	1 457,00 руб.
Гермомешок Helios 70 л (HS-DB-703865-H)	1 457,00 руб.
Гермомешок Helios 80 л HS-GB-80-BB	2 453,00 руб.
Гермомешок Helios 90 л (HS-DB-9033125-B)	1 633,00 руб.
Гермомешок Helios 90 л (HS-DB-9033125-H)	1 633,00 руб.
Гермомешок Helios 90 л (HS-DB-9033125-Y)	1 633,00 руб.
Гермомешок Helios 90 л (HS-DB-903885)	1 633,00 руб.
Гермомешок Tramp 100л TRA-109 прозрачный	1 587,00 руб.
Гермомешок Tramp 100л TRA-210 оливковый	1 126,00 руб.
Гермомешок Tramp 20 л нейлон красный TRA-102	767,00 руб.
Гермомешок Tramp 20л TRA-067 оранжевый	1 060,00 руб.
Гермомешок Tramp 50 л нейлон красный TRA-103	1 167,00 руб.
Гермомешок Tramp 50 л ПВХ оранжевый TRA-068	1 394,00 руб.
Гермомешок Tramp 50л TRA-107 прозрачный	1 153,00 руб.
Гермомешок Tramp 70 л нейлон красный TRA-104	1 300,00 руб.
Гермомешок Tramp 70 л ПВХ оранжевый TRA-069	1 834,00 руб.
Гермомешок Tramp 70л TRA-108 прозрачный	1 327,00 руб.
"""